## 1️⃣ Import des datasets

In [2]:
import pandas as pd
from pathlib import Path

In [3]:
# 📂 Chargement des fichiers nettoyés
df_fin = pd.read_csv("../data/df_mm_financial_clean.csv")
df_sto = pd.read_csv("../data/df_mm_stores_clean.csv")

In [4]:
print("✅ Fichiers chargés :")
print(f"Finance : {df_fin.shape}")
print(f"Stores  : {df_sto.shape}")


✅ Fichiers chargés :
Finance : (33856, 15)
Stores  : (33855, 9)


In [5]:
# 🔍 Vérifions la clé commune
print("\nClés communes potentielles :")
print(set(df_fin.columns) & set(df_sto.columns))


Clés communes potentielles :
{'reportid'}


In [6]:
# 💡 Clé de jointure -> 'reportid'
key = "reportid"
if key not in df_fin.columns or key not in df_sto.columns:
    raise KeyError(f"La clé '{key}' n'existe pas dans les deux fichiers !")

In [7]:
# 🔑 Vérification de doublons sur la clé
print("\nDoublons sur la clé :")
print(f"Finance : {df_fin[key].duplicated().sum()}")
print(f"Stores  : {df_sto[key].duplicated().sum()}")


Doublons sur la clé :
Finance : 1
Stores  : 0


In [8]:
# 🔗 Fusion (INNER JOIN)
df_join = pd.merge(
    df_fin,
    df_sto,
    on=key,
    how="inner",
    suffixes=("_fin", "_sto")
)

print("\n✅ Fusion réussie !")
print(f"Dimensions finales : {df_join.shape}")

display(df_join.head(5))


✅ Fusion réussie !
Dimensions finales : (33856, 23)


,unit_price,unit_cost,sold_quantity,sales,cogs,marketing,total_expenses,pre-sales_inventory,profit,margin,...,expected_margin,reportid,quarter,storeid,state,position,market,prodid,product,type
0,4.5,2.69,666.0,2997.0,1791.54,226.44,2017.98,948.0,979.02,NaN,...,48.38,JB578CJ,NaN,Los Angeles - Sunset Boulevard,California,West,Large Market,1_Co,Amaretto,Coffee
1,4.5,2.69,1161.0,5224.5,3123.09,394.74,3517.83,2012.0,1706.67,24.00,...,46.03,UZ650HV,2023-04-01,Los Angeles - Sunset Boulevard,California,West,Large Market,NaN,Amaretto,Coffee
2,4.5,2.69,409.0,1840.5,1100.21,139.06,1239.27,581.0,601.23,25.56,...,42.86,VB278N7,2023-07-01,Los Angeles - Sunset Boulevard,California,West,Large Market,1_Co,Amaretto,Coffee
3,4.5,2.69,1071.0,4819.5,2880.99,364.14,NaN,1629.0,1574.37,25.06,...,48.60,TL8282Y,2023-10-01,Los Angeles - Sunset Boulevard,California,West,Large Market,1_Co,Amaretto,Coffee
4,4.5,2.69,735.0,3307.5,1977.15,249.90,2227.05,1217.0,1080.45,24.39,...,48.38,FB512II,2023-01-01,Los Angeles - Hollywood Boulevard,California,West,Large Market,1_Co,Amaretto,Coffee


In [9]:
# 💾 Export du fichier fusionné
import os

os.makedirs("../data", exist_ok=True)  # Crée le dossier si besoin

output_path_join = "../data/df_join.csv"
df_join.to_csv(output_path_join, index=False, encoding="utf-8-sig")

print(f"✅ Fichier fusionné exporté avec succès : {output_path_join}")
print(f"Taille du dataset : {df_join.shape}")

✅ Fichier fusionné exporté avec succès : ../data/df_join.csv
Taille du dataset : (33856, 23)


In [10]:
# ===========================================
# 🧹 Traitement raisonné des valeurs manquantes
# ===========================================

# 1️⃣ Créer une copie de sécurité
df_join_clean = df_join.copy()

# 2️⃣ Aperçu global des valeurs manquantes
missing_summary = df_join_clean.isna().sum().sort_values(ascending=False)
print("📊 Valeurs manquantes par colonne :\n", missing_summary[missing_summary > 0])
print(f"\nTotal de valeurs manquantes : {df_join_clean.isna().sum().sum()}")

# 3️⃣ Suppression uniquement des lignes où les colonnes clés sont manquantes
key_columns = ["reportid", "sales", "storeid", "product"]  # adapte selon ton dataset
rows_before = df_join_clean.shape[0]

df_join_clean = df_join_clean.dropna(subset=key_columns)

rows_after = df_join_clean.shape[0]
print(f"\n🗑️ Lignes supprimées (valeurs manquantes dans les colonnes clés) : {rows_before - rows_after}")

# 4️⃣ Imputation des valeurs manquantes restantes

## Numériques : moyenne ou médiane selon la distribution
num_cols = df_join_clean.select_dtypes(include=['float64', 'int64']).columns
for col in num_cols:
    if df_join_clean[col].isna().sum() > 0:
        skew = df_join_clean[col].skew()
        if abs(skew) > 1:
            df_join_clean[col].fillna(df_join_clean[col].median(), inplace=True)
            print(f"🟠 Colonne '{col}' → imputée par la médiane (skew={skew:.2f})")
        else:
            df_join_clean[col].fillna(df_join_clean[col].mean(), inplace=True)
            print(f"🟢 Colonne '{col}' → imputée par la moyenne (skew={skew:.2f})")

## Catégorielles : modalité la plus fréquente
cat_cols = df_join_clean.select_dtypes(include=['object']).columns
for col in cat_cols:
    if df_join_clean[col].isna().sum() > 0:
        mode_val = df_join_clean[col].mode()[0]
        df_join_clean[col].fillna(mode_val, inplace=True)
        print(f"🔵 Colonne '{col}' → imputée par la valeur la plus fréquente : '{mode_val}'")

# 5️⃣ Vérification finale
missing_after = df_join_clean.isna().sum().sum()
print(f"\n✅ Nettoyage terminé — valeurs manquantes restantes : {missing_after}")
if missing_after == 0:
    print("🎉 Toutes les valeurs manquantes ont été gérées avec succès.")
else:
    print("⚠️ Il reste quelques valeurs manquantes, à vérifier manuellement.")


📊 Valeurs manquantes par colonne :
 margin                 3222
product                3095
expected_margin        2956
sales                  2941
pre-sales_inventory    2897
type                   2834
expected_sales         2639
total_expenses         2472
market                 2067
prodid                 1752
unit_price             1716
storeid                1513
cogs                   1283
sold_quantity          1181
quarter                1176
position               1143
unit_cost              1093
profit                  960
expected_profit         860
expected_cogs           732
marketing               703
state                   667
dtype: int64

Total de valeurs manquantes : 39902

🗑️ Lignes supprimées (valeurs manquantes dans les colonnes clés) : 7046
🟢 Colonne 'unit_price' → imputée par la moyenne (skew=0.27)
🟠 Colonne 'unit_cost' → imputée par la médiane (skew=1.13)
🟠 Colonne 'sold_quantity' → imputée par la médiane (skew=1.73)
🟠 Colonne 'cogs' → imputée par la médiane (

C:\Users\ryanm\AppData\Local\Temp\ipykernel_16196\1206409812.py:33: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_join_clean[col].fillna(df_join_clean[col].mean(), inplace=True)
C:\Users\ryanm\AppData\Local\Temp\ipykernel_16196\1206409812.py:30: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves

In [12]:
df_join_clean.shape


(26810, 23)